# Part 2: Visualizing geo-data

**Select a couple of months of data (for example, June-July 2016) for 'DRUG/NARCOTIC' and the same couple of month for 'LARCENY/THEFT'. Note: There is an issue with folium and plotting lots of points, so if you have trouble plotting simply select a shorter time period; you should be able to plot a couple of thousand points.**

In [102]:
import pandas as pd
import folium

In [103]:
data = pd.read_csv("/Users/ainc/Documents/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv",
                   parse_dates=['Date'])

Define the time period to work with. In order to overcome folium limitations on how many points can be plotted on a scatter plot, we decieded to focus on crimes that took place in the first 10 days of June 2017.

In [104]:
YEAR = 2017
MONTH = [6]
BEFORE_DAY = 10

Define a function to filter the data from the original dataset:

In [105]:
def filter_crimes(data, category, year, month, before_day):
    crimes = data[(data['Category'] == category)]
    crimes = crimes[(crimes['Date'].dt.year == year) & (crimes['Date'].dt.month.isin(month)) & (crimes['Date'].dt.day < before_day)]
    return crimes

**Draw a little circle for each arrest in the category 'DRUG/NARCOTIC' for your time period. We can call this a kind of visualization a scatter plot**

Filter data from the original dataset:

In [106]:
drug_crimes = filter_crimes(data, 'DRUG/NARCOTIC', YEAR, MONTH, BEFORE_DAY)
print(f'No. of drug crimes: {len(drug_crimes)}')

No. of drug crimes: 61


Construct the map:

In [107]:
m = folium.Map([37.77919, -122.41914], zoom_start=13)
for index, row in drug_crimes.iterrows():
    folium.CircleMarker([row['Y'], row['X']],
                        popup=str(row['IncidntNum']),
                        radius=5,
                        color="blue",
                       ).add_to(m)
m

**Draw a little circle for each arrest in the category 'LARCENY/THEFT' for the time period.**

Filter data from the original dataset:

In [108]:
theft_crimes = filter_crimes(data, 'LARCENY/THEFT', YEAR, MONTH, BEFORE_DAY)
print(f'No. of theft crimes: {len(theft_crimes)}')

No. of theft crimes: 1126


Construct the map:

In [109]:
m = folium.Map([37.77919, -122.41914], zoom_start=13)
for index, row in theft_crimes.iterrows():
    folium.CircleMarker([row['Y'], row['X']],
                        popup=str(row['IncidntNum']),
                        radius=5,
                        color="red",
                       ).add_to(m)
m

**Now, let's play with heatmaps.**

In [110]:
from folium import plugins

Define function to construct the heatmap. A specific type of tiles has been selected to show a black and white map to highlight the colors of the data shown on top of it.

In [111]:
def getHeatMap(data):
    m = folium.Map([37.77919, -122.41914], tiles='stamentoner', zoom_start=13)
    coordinates = data[['Y', 'X']].values
    m.add_child(plugins.HeatMap(coordinates, radius=15))
    return m

**Create a heatmap of all arrests for 'DRUG/NARCOTIC' during June-July 2016 (with the heatmap you should be able to use the full period). Play with parameters to get plots you like**

In [112]:
drug_crimes = filter_crimes(data, 'DRUG/NARCOTIC', 2016, [6,7], 32)

In [113]:
getHeatMap(drug_crimes)

**Create a heatmap of all arrests for 'LARCENY/THEFT' during June-July 2016 (with the heatmap you should be able to use the full period).**

In [114]:
theft_crimes = filter_crimes(data, 'LARCENY/THEFT', 2016, [6,7], 32)

In [115]:
getHeatMap(theft_crimes)

**Comment on the differences. What can you see using the scatter-plots that you can't see using the heatmaps? And vice versa: what does the heatmaps help you see that's difficult to distinguish in the scatter-plots?**

The drugs/narcotic crime is very much limited to a specific area and has few high density places within that.
The larceny/theft crime, on the other hand, covers a wider area and has more high density regions. A couple in the same area as the drugs/narcotic one, and others more north and west.
From the heatmap, it looks like theft is a more common crime and affecting a wider area of San Francisco, whereas the drug crimes are less and limited to specific places.

Heatmaps are very intiutive tool to detect density differences in different areas and to get a direct visual feedback on how vast a crime affected area is and where the hot points are. This cannot be achieved with the same level of intuitiveness with scatter plots. 
Scatter plots, on the other hand, give a realistic idea of how many occurrences happened in each zone by plotting each occurrence. This allows to have a high degree of details and fidelity over the position and amount of crimes happened. Heatmaps however hide this sort of details when plotting densities. 

Scatter plot:
* **Pros:**
    * This plot allows to see all the occurrences spread on the plotted area. This means that it gives a realistic idea of how many occurrences happened in each zone. 
* **Cons:**
    * It is difficult to get a precise idea of how the distribution and concentration look like, over the area. High density areas tend to look like an agglomerate of color, because of all the circles in the same area. It is though difficult to distinguish the different agglomerate in terms of which ones are more dense than others.

Heatmaps:
* **Pros:**
    * These maps highlight the concentration of crimes in specific areas and the distribution of the different densities over the map. It is then easy to compare the differents zones' densities just by comparing their color. 
* **Cons:**
    * Hide away the real number of occurrences. It is not easy to determine how many crimes a high density zone contain.


**Comment on the effect on the various parameters for the heatmaps. How do they change the picture?**

From the official documentation, these are the parameter which can be passed to the heatmap:
* data (list of points of the form [lat, lng] or [lat, lng, weight]) – The points you want to plot. You can also provide a numpy.array of shape (n,2) or (n,3).
* name (string, default None) – The name of the Layer, as it will appear in LayerControls.
* min_opacity (default 1.) – The minimum opacity the heat will start at.
* max_zoom (default 18) – Zoom level where the points reach maximum intensity (as intensity scales with zoom), equals  maxZoom of the map by default
* max_val (float, default 1.) – Maximum point intensity
* radius (int, default 25) – Radius of each “point” of the heatmap
* blur (int, default 15) – Amount of blur
* gradient (dict, default None) – Color gradient config. e.g. {0.4: ‘blue’, 0.65: ‘lime’, 1: ‘red’}
* overlay (bool, default True) – Adds the layer as an optional overlay (True) or the base layer (False).
* control (bool, default True) – Whether the Layer will be included in LayerControls.
* show (bool, default True) – Whether the layer will be shown on opening (only for overlays).

#### Radius and Blur parameter

By reducing both the radius and the blurriness, it is possible to achieve a heatmap which recalls the scatter plot one: with sharp circles for each occurrence. The color of the circles, however, still reppresent the concentration of crimes, allowing us to have more information than a scatter plot.

In [116]:
def getHeatMap(data):
    m = folium.Map([37.77919, -122.41914], tiles='stamentoner', zoom_start=13)
    coordinates = data[['Y', 'X']].values
    m.add_child(plugins.HeatMap(coordinates, radius=10, blur=0.1))
    return m

In [117]:
drug_crimes = filter_crimes(data, 'DRUG/NARCOTIC', 2016, [6,7], 32)
getHeatMap(drug_crimes)

#### Blur parameter

This parameter determines how blurry the circles will be on the map. Increasing it makes the circle , whereas decreasing it allows to draw sharper circles. By reducing the blurrines, the achieved effect is to a more precise reppresentation of the occurrences, more similar to a scatter plot.

In [118]:
def getHeatMap(data):
    m = folium.Map([37.77919, -122.41914], tiles='stamentoner', zoom_start=13)
    coordinates = data[['Y', 'X']].values
    m.add_child(plugins.HeatMap(coordinates, radius=10, blur=10))
    return m

In [119]:
drug_crimes = filter_crimes(data, 'DRUG/NARCOTIC', 2016, [6,7], 32)
getHeatMap(drug_crimes)

#### Min_opacity parameter

In [120]:
def getHeatMap(data):
    m = folium.Map([37.77919, -122.41914], tiles='stamentoner', zoom_start=13)
    coordinates = data[['Y', 'X']].values
    m.add_child(plugins.HeatMap(coordinates, min_opacity=0.01))
    return m

In [121]:
drug_crimes = filter_crimes(data, 'DRUG/NARCOTIC', 2016, [6,7], 32)
getHeatMap(drug_crimes)